In [11]:
import torch
import torchvision

from argparse import ArgumentParser
from pytorch_lightning import LightningModule, Trainer, LightningDataModule
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchmetrics.functional import accuracy
from einops import rearrange
from torch import nn
from torchvision.datasets.cifar import CIFAR10

In [12]:
class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False):
        super().__init__()
        assert dim % num_heads == 0, 'dim should be divisible by num_heads'
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.proj = nn.Linear(dim, dim)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv.unbind(0)   # make torchscript happy (cannot use tensor as tuple)

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)

        return x

In [13]:
class Mlp(nn.Module):
    """ MLP as used in Vision Transformer, MLP-Mixer and related networks
    """
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
      
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)
        return x

In [14]:
class Block(nn.Module):

    def __init__(
            self, dim, num_heads, mlp_ratio=4., qkv_bias=False, 
            act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(dim, num_heads=num_heads, qkv_bias=qkv_bias) 
        self.norm2 = norm_layer(dim)
        self.mlp = Mlp(in_features=dim, hidden_features=int(dim * mlp_ratio), act_layer=act_layer) 
   

    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x

In [15]:
class Transformer(nn.Module):
    def __init__(self, dim, num_heads, num_blocks, mlp_ratio=4., qkv_bias=False,  
                 act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.blocks = nn.ModuleList([Block(dim, num_heads, mlp_ratio, qkv_bias, 
                                     act_layer, norm_layer) for _ in range(num_blocks)])

    def forward(self, x):
        for block in self.blocks:
            x = block(x)
        return x

In [16]:
def init_weights_vit_timm(module: nn.Module):
    """ ViT weight initialization, original timm impl (for reproducibility) """
    if isinstance(module, nn.Linear):
        nn.init.trunc_normal_(module.weight, mean=0.0, std=0.02)
        if module.bias is not None:
            nn.init.zeros_(module.bias)
    elif hasattr(module, 'init_weights'):
        module.init_weights()

In [17]:
class LitTransformer(LightningModule):
    def __init__(self, num_classes=10, lr=0.001, max_epochs=30, depth=12, embed_dim=64,
                 head=4, patch_dim=192, seqlen=16, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.encoder = Transformer(dim=embed_dim, num_heads=head, num_blocks=depth, mlp_ratio=4.,
                                   qkv_bias=False, act_layer=nn.GELU, norm_layer=nn.LayerNorm)
        self.embed = torch.nn.Linear(patch_dim, embed_dim)

        self.fc = nn.Linear(seqlen * embed_dim, num_classes)
        self.loss = torch.nn.CrossEntropyLoss()
        
        self.reset_parameters()


    def reset_parameters(self):
        init_weights_vit_timm(self)
    

    def forward(self, x):
        # Linear projection
        x = self.embed(x)
            
        # Encoder
        x = self.encoder(x)
        x = x.flatten(start_dim=1)

        # Classification head
        x = self.fc(x)
        return x
    
    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=self.hparams.lr)
        # this decays the learning rate to 0 after max_epochs using cosine annealing
        scheduler = CosineAnnealingLR(optimizer, T_max=self.hparams.max_epochs)
        return [optimizer], [scheduler]

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        return loss
    

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        acc = accuracy(y_hat, y)
        return {"y_hat": y_hat, "test_loss": loss, "test_acc": acc}

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        avg_acc = torch.stack([x["test_acc"] for x in outputs]).mean()
        self.log("test_loss", avg_loss, on_epoch=True, prog_bar=True)
        self.log("test_acc", avg_acc*100., on_epoch=True, prog_bar=True)

    def validation_step(self, batch, batch_idx):
        return self.test_step(batch, batch_idx)

    def validation_epoch_end(self, outputs):
        return self.test_epoch_end(outputs)



In [ ]:

# a lightning data module for cifar 10 dataset
class LitCifar10(LightningDataModule):
    def __init__(self, batch_size=32, num_workers=32, patch_num=4, **kwargs):
        super().__init__()
        self.batch_size = batch_size
        self.patch_num = patch_num
        self.num_workers = num_workers

    def prepare_data(self):
        self.train_set = CIFAR10(root='./data', train=True,
                                 download=True, transform=torchvision.transforms.ToTensor())
        self.test_set = CIFAR10(root='./data', train=False,
                                download=True, transform=torchvision.transforms.ToTensor())

    def collate_fn(self, batch):
        x, y = zip(*batch)
        x = torch.stack(x, dim=0)
        y = torch.LongTensor(y)
        x = rearrange(x, 'b c (p1 h) (p2 w) -> b (p1 p2) (c h w)', p1=self.patch_num, p2=self.patch_num)
        return x, y

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_set, batch_size=self.batch_size, 
                                        shuffle=True, collate_fn=self.collate_fn,
                                        num_workers=self.num_workers)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.test_set, batch_size=self.batch_size, 
                                        shuffle=False, collate_fn=self.collate_fn,
                                        num_workers=self.num_workers)

    def val_dataloader(self):
        return self.test_dataloader()




In [ ]:

def get_args():
    parser = ArgumentParser(description='PyTorch Transformer')
    parser.add_argument('--depth', type=int, default=12, help='depth')
    parser.add_argument('--embed_dim', type=int, default=64, help='embedding dimension')
    parser.add_argument('--num_heads', type=int, default=4, help='num_heads')

    parser.add_argument('--patch_num', type=int, default=8, help='patch_num')
    parser.add_argument('--kernel_size', type=int, default=3, help='kernel size')
    parser.add_argument('--batch_size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: )')
    parser.add_argument('--max-epochs', type=int, default=30, metavar='N',
                        help='number of epochs to train (default: 0)')
    parser.add_argument('--lr', type=float, default=0.001, metavar='LR',
                        help='learning rate (default: 0.0)')

    parser.add_argument('--accelerator', default='gpu', type=str, metavar='N')
    parser.add_argument('--devices', default=1, type=int, metavar='N')
    parser.add_argument('--dataset', default='cifar10', type=str, metavar='N')
    parser.add_argument('--num_workers', default=4, type=int, metavar='N')
    args = parser.parse_args("")
    return args

In [18]:
if __name__ == "__main__":
    args = get_args()

    datamodule = LitCifar10(batch_size=args.batch_size,
                            patch_num=args.patch_num, 
                            num_workers=args.num_workers * args.devices)
    datamodule.prepare_data()

    data = iter(datamodule.train_dataloader()).next()
    patch_dim = data[0].shape[-1]
    seqlen = data[0].shape[-2]
    print("Embed dim:", args.embed_dim)
    print("Patch size:", 32 // args.patch_num)
    print("Sequence length:", seqlen)


    model = LitTransformer(num_classes=10, lr=args.lr, epochs=args.max_epochs, 
                           depth=args.depth, embed_dim=args.embed_dim, head=args.num_heads,
                           patch_dim=patch_dim, seqlen=seqlen,)

    trainer = Trainer(accelerator=args.accelerator, devices=args.devices,
                      max_epochs=args.max_epochs, precision=16 if args.accelerator == 'gpu' else 32,)
    trainer.fit(model, datamodule=datamodule)

Files already downloaded and verified
Files already downloaded and verified


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Embed dim: 64
Patch size: 4
Sequence length: 64
Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params
---------------------------------------------
0 | encoder | Transformer      | 597 K 
1 | embed   | Linear           | 3.1 K 
2 | fc      | Linear           | 41.0 K
3 | loss    | CrossEntropyLoss | 0     
---------------------------------------------
641 K     Trainable params
0         Non-trainable params
641 K     Total params
1.283     Total estimated model params size (MB)


Epoch 29: 100%|██████████| 939/939 [00:19<00:00, 47.40it/s, loss=9.78e-05, v_num=18, test_loss=2.250, test_acc=70.40]


CUSTOM LINES

In [19]:

data = iter(datamodule.train_dataloader()).next()

In [26]:
i = 0
data[0][i]

tensor([[0.3020, 0.2902, 0.2863,  ..., 0.3255, 0.3922, 0.3647],
        [0.3333, 0.3373, 0.3882,  ..., 0.2980, 0.1647, 0.0510],
        [0.1255, 0.1020, 0.1373,  ..., 0.0549, 0.1255, 0.1137],
        ...,
        [0.7765, 0.6549, 0.4235,  ..., 0.2353, 0.3843, 0.5216],
        [0.4588, 0.4078, 0.3216,  ..., 0.2745, 0.3137, 0.4039],
        [0.4196, 0.4784, 0.4706,  ..., 0.4706, 0.4471, 0.4118]])

In [28]:
data[1][i]

tensor(3)

In [29]:
data[0][i].shape

torch.Size([64, 48])

In [30]:
data[0]


tensor([[[0.3020, 0.2902, 0.2863,  ..., 0.3255, 0.3922, 0.3647],
         [0.3333, 0.3373, 0.3882,  ..., 0.2980, 0.1647, 0.0510],
         [0.1255, 0.1020, 0.1373,  ..., 0.0549, 0.1255, 0.1137],
         ...,
         [0.7765, 0.6549, 0.4235,  ..., 0.2353, 0.3843, 0.5216],
         [0.4588, 0.4078, 0.3216,  ..., 0.2745, 0.3137, 0.4039],
         [0.4196, 0.4784, 0.4706,  ..., 0.4706, 0.4471, 0.4118]],

        [[0.3843, 0.4118, 0.4039,  ..., 0.2902, 0.3137, 0.3294],
         [0.3843, 0.4471, 0.4431,  ..., 0.3529, 0.3412, 0.3098],
         [0.4353, 0.4588, 0.4941,  ..., 0.3529, 0.3804, 0.3529],
         ...,
         [0.5490, 0.5647, 0.5176,  ..., 0.5216, 0.4627, 0.4000],
         [0.3922, 0.4118, 0.4353,  ..., 0.3922, 0.3725, 0.3686],
         [0.6353, 0.6667, 0.6588,  ..., 0.4235, 0.3843, 0.3647]],

        [[0.5176, 0.5176, 0.5569,  ..., 0.3922, 0.4549, 0.5608],
         [0.5804, 0.5765, 0.5843,  ..., 0.5765, 0.5765, 0.5882],
         [0.6157, 0.6196, 0.6235,  ..., 0.5765, 0.6196, 0.

In [31]:
data[1]


tensor([3, 5, 1, 3, 0, 9, 3, 6, 1, 4, 2, 9, 9, 7, 6, 0, 4, 2, 7, 6, 8, 5, 7, 4,
        9, 5, 7, 1, 1, 7, 7, 9, 7, 0, 4, 7, 0, 6, 6, 0, 7, 8, 5, 3, 4, 3, 0, 6,
        0, 3, 3, 7, 3, 0, 4, 6, 7, 3, 6, 6, 2, 1, 7, 0])